<h1><center>Basic TF Keras Starter</center></h1>

Based on this excellent work by Stanley Zheng:
https://www.kaggle.com/stanleyjzheng/baseline-nn-with-k-folds


### Import Libraries

In [1]:

import numpy as np 
import pandas as pd 
import os

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
 
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


### Read Data

In [2]:
test_df = pd.read_csv('../input/lish-moa/test_features.csv')
train_df = pd.read_csv('../input/lish-moa/train_features.csv')
train_target_df = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
sub = pd.read_csv('../input/lish-moa/sample_submission.csv')

target_cols = train_target_df.columns[1:]
N_TARGETS = len(target_cols)

In [3]:
train_df.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [4]:
sub.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,id_001897cda,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,id_002429b5b,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,id_00276f245,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,id_0027f1083,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


### Basic Setup and Helpers

In [5]:
SEED = 1234
EPOCHS = 28
BATCH_SIZE = 128
FOLDS = 5
REPEATS = 5
LR = 0.0005
N_TARGETS = len(target_cols)

In [6]:
def seed_everything(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [7]:
def multi_log_loss(y_true, y_pred):
    losses = []
    for col in y_true.columns:
        losses.append(log_loss(y_true.loc[:, col], y_pred.loc[:, col]))
    return np.mean(losses)

### Encode Categoricals to Binary

In [8]:
def preprocess_df(data):
    data['cp_type'] = (data['cp_type'] == 'trt_cp').astype(int)
    data['cp_dose'] = (data['cp_dose'] == 'D2').astype(int)
    return data

In [9]:
x_train = preprocess_df(train_df.drop(columns="sig_id"))
x_test =preprocess_df(test_df.drop(columns="sig_id"))
y_train = train_target_df.drop(columns="sig_id")
N_FEATURES = x_train.shape[1]

### Define Model Architecture

In [10]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATURES),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.4),
    #tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),  
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

### Main CV and Model Training Function

In [11]:
def build_train(resume_models = None, repeat_number = 0, folds = 5, skip_folds = 0):
    
    models = []
    oof_preds = y_train.copy()
    

    kfold = KFold(folds, shuffle = True)
    for fold, (train_ind, val_ind) in enumerate(kfold.split(x_train)):
        print('\n')
        print('-'*50)
        print(f'Training fold {fold + 1}')
        
        cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'binary_crossentropy', factor = 0.4, patience = 2, verbose = 1, min_delta = 0.0001, mode = 'auto')
        checkpoint_path = f'repeat:{repeat_number}_Fold:{fold}.hdf5'
        cb_checkpt = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True, save_weights_only = True, mode = 'min')

        model = create_model()
        model.fit(x_train.values[train_ind],
              y_train.values[train_ind],
              validation_data=(x_train.values[val_ind], y_train.values[val_ind]),
              callbacks = [cb_lr_schedule, cb_checkpt],
              epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2
             )
        model.load_weights(checkpoint_path)
        oof_preds.loc[val_ind, :] = model.predict(x_train.values[val_ind])
        models.append(model)

    return models, oof_preds

In [12]:
models = []
oof_preds = []
# seed everything
seed_everything(SEED)
for i in range(REPEATS):
    m, oof = build_train(repeat_number = i, folds=FOLDS)
    models = models + m
    oof_preds.append(oof)





--------------------------------------------------
Training fold 1
Epoch 1/28
149/149 - 2s - loss: 0.5639 - accuracy: 0.0232 - val_loss: 0.2260 - val_accuracy: 0.0609
Epoch 2/28
149/149 - 1s - loss: 0.1001 - accuracy: 0.0493 - val_loss: 0.0404 - val_accuracy: 0.0615
Epoch 3/28
149/149 - 1s - loss: 0.0337 - accuracy: 0.0655 - val_loss: 0.0256 - val_accuracy: 0.0980
Epoch 4/28
149/149 - 1s - loss: 0.0242 - accuracy: 0.0708 - val_loss: 0.0209 - val_accuracy: 0.0796
Epoch 5/28
149/149 - 2s - loss: 0.0212 - accuracy: 0.0827 - val_loss: 0.0198 - val_accuracy: 0.0825
Epoch 6/28
149/149 - 1s - loss: 0.0198 - accuracy: 0.0882 - val_loss: 0.0188 - val_accuracy: 0.0846
Epoch 7/28
149/149 - 1s - loss: 0.0189 - accuracy: 0.0925 - val_loss: 0.0182 - val_accuracy: 0.1157
Epoch 8/28
149/149 - 1s - loss: 0.0183 - accuracy: 0.0967 - val_loss: 0.0177 - val_accuracy: 0.0947
Epoch 9/28
149/149 - 1s - loss: 0.0179 - accuracy: 0.1039 - val_loss: 0.0173 - val_accuracy: 0.1027
Epoch 10/28
149/149 - 1s - loss

In [13]:
mean_oof_preds = y_train.copy()
mean_oof_preds.loc[:, target_cols] = 0
for i, p in enumerate(oof_preds):
    print(f"Repeat {i + 1} OOF Log Loss: {multi_log_loss(y_train, p)}")
    mean_oof_preds.loc[:, target_cols] += p[target_cols]

mean_oof_preds.loc[:, target_cols] /= len(oof_preds)
print(f"Mean OOF Log Loss: {multi_log_loss(y_train, mean_oof_preds)}")
mean_oof_preds.loc[x_train['cp_type'] == 0, target_cols] = 0
print(f"Mean OOF Log Loss (ctl adjusted): {multi_log_loss(y_train, mean_oof_preds)}")

Repeat 1 OOF Log Loss: 0.015290353340956927
Repeat 2 OOF Log Loss: 0.015250066151224798
Repeat 3 OOF Log Loss: 0.015210289656689578
Repeat 4 OOF Log Loss: 0.015247687109763852
Repeat 5 OOF Log Loss: 0.015251954843501165
Mean OOF Log Loss: 0.014882315782138321
Mean OOF Log Loss (ctl adjusted): 0.01483389486878573


### Make Test Predictions and Save Submission

In [14]:
test_preds = sub.copy()
test_preds[target_cols] = 0
for model in models:
    test_preds.loc[:,target_cols] += model.predict(x_test)
test_preds.loc[:,target_cols] /= len(models)
test_preds.loc[x_test['cp_type'] == 0, target_cols] = 0
test_preds.to_csv('submission.csv', index=False)